# 데이터 불러오기

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

import seaborn as sns
import matplotlib.pyplot as plt

df_train = pd.read_csv('data/train.csv')
df29 = pd.read_csv('./data/submission_29.csv')
df_test = pd.read_csv('./data/test_with_label.csv')

# True no_relation -> Pred else [id]

In [19]:
no_but_else = [] # save id 
for i in range(df29.shape[0]):
    if df_test.iloc[i,:]['label'] == 'no_relation' and df29.iloc[i,:]['pred_label'] == 'else':
        no_but_else.append(i)
len(no_but_else)

989

# True no_relation -> Pred else -> re_Pred no_relation

In [23]:
## under sampling data로 학습시킨 모델에
## test_data.csv에서 실제 no_relation이었지만, 처음에 binary에서 else로 예측했다가 나중에 두 번째 모형에서 no_relation으로 예측한 개수

# load new submission
df_sub = pd.read_csv("./data/submission_no_else_no.csv") # 7765

else2no = [] # save id 
for i in no_but_else:
    if df_sub.iloc[i,:]['pred_label'] == 'no_relation':    # no_but_else에는 no_relation을 else로 잘못 예측한 id들이 들어가있음
        else2no.append(i)
len(else2no)

326

# F1_score 계산
(221130)

어제는 내가 만든 두 번째 모형이 29개의 label만 분류를 했지만,<br>
지금 모형은 기존의 30개 label을 전부 예측할 수 있는 모형.

어제와의 차이점이 존재한다.

어제 모형은 실제 no_relation을 else로 예측한 애들을 다시 no_relation으로 알맞게 예측할 수 없었음(29개만 하기 때문에)

## 30 label ver.

inference를 test_data.csv 로 했을 때.

In [77]:
df_infer = pd.read_csv("./data/submission_train_else.csv")
sum(df_infer['pred_label'] == 'no_relation')

0

In [78]:
df_infer

,id,pred_label,probs
0,0,org:product,"[0.00014809485583100468, 4.521029404713772e-05..."
1,1,per:title,"[0.00010074729652842507, 4.3126226955791935e-0..."
2,2,org:dissolved,"[0.0007704586605541408, 0.0002529222401790321,..."
3,3,per:other_family,"[0.000770598475355655, 0.0006429736968129873, ..."
4,4,org:top_members/employees,"[1.50126052176347e-05, 0.9995294809341431, 2.6..."
...,...,...,...
7760,7760,org:place_of_headquarters,"[4.12692861573305e-05, 0.00010937532351817936,..."
7761,7761,org:number_of_employees/members,"[0.0022659816313534975, 0.0010699615813791752,..."
7762,7762,org:top_members/employees,"[1.494631123932777e-05, 0.9995831847190857, 1...."
7763,7763,per:date_of_birth,"[0.00022600110969506204, 9.997283632401377e-05..."


In [79]:
df_sub = pd.read_csv("./data/submission_29.csv")
id_norelation = []

for i in range(df_sub.shape[0]):
    if df_sub.iloc[i,:]['pred_label'] == 'no_relation':
        id_norelation.append(i)

df_infer = pd.read_csv("./data/submission_train_else.csv")
label = []
for i in range(df_sub.shape[0]):
    if i in id_norelation:
        label.append('no_relation')
    else:
        label.append(df_infer.iloc[i,:]['pred_label'])

df_infer.drop(labels = 'pred_label', axis=1, inplace=True)
df_infer = pd.concat([df_infer, pd.Series(label)], axis = 1)
df_infer.columns = ['id', 'probs', 'pred_label']
df_infer = df_infer[['id','pred_label','probs']]

# df_infer
sum(df_infer['pred_label'] == 'no_relation')

4350

In [80]:
l_dic = {'no_relation': 0, 'org:top_members/employees': 1, 'org:members': 2, 'org:product': 3, 'per:title': 4, 'org:alternate_names': 5, 'per:employee_of': 6, 'org:place_of_headquarters': 7, 'per:product': 8, 'org:number_of_employees/members': 9, 'per:children': 10, 
         'per:place_of_residence': 11, 'per:alternate_names': 12, 'per:other_family': 13, 'per:colleagues': 14, 'per:origin': 15, 'per:siblings': 16, 'per:spouse': 17, 'org:founded': 18, 'org:political/religious_affiliation': 19, 'org:member_of': 20, 
         'per:parents': 21, 'org:dissolved': 22, 'per:schools_attended': 23, 'per:date_of_death': 24, 'per:date_of_birth': 25, 'per:place_of_birth': 26, 'per:place_of_death': 27, 'org:founded_by': 28, 'per:religion': 29}

true = list(df_test['label'])
pred = list(df_infer['pred_label'])

true = [l_dic[label] for label in true]
pred = [l_dic[label] for label in pred]

## 29 label ver.

inference를 test_else.csv 로 했을 때

### f1 score

In [62]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import sklearn
def klue_re_micro_f1(preds, labels):
    """KLUE-RE micro f1 (except no_relation)"""
    label_list = ['no_relation', 'org:top_members/employees', 'org:members',
       'org:product', 'per:title', 'org:alternate_names',
       'per:employee_of', 'org:place_of_headquarters', 'per:product',
       'org:number_of_employees/members', 'per:children',
       'per:place_of_residence', 'per:alternate_names',
       'per:other_family', 'per:colleagues', 'per:origin', 'per:siblings',
       'per:spouse', 'org:founded', 'org:political/religious_affiliation',
       'org:member_of', 'per:parents', 'org:dissolved',
       'per:schools_attended', 'per:date_of_death', 'per:date_of_birth',
       'per:place_of_birth', 'per:place_of_death', 'org:founded_by',
       'per:religion']
    no_relation_label_idx = label_list.index("no_relation")
    label_indices = list(range(len(label_list)))
    label_indices.remove(no_relation_label_idx)
    return sklearn.metrics.f1_score(labels, preds, average="micro", labels=label_indices) * 100.0

In [81]:
klue_re_micro_f1(pred, true)

63.612765307680554

# 맞춘 개수 count

## 30 label ver.

In [82]:
pred_else = [] # binary에서 'else'로 예측한 id
for i in range(df29.shape[0]):
    if df29.iloc[i,:]['pred_label'] == 'else':
        pred_else.append(i)
len(pred_else)

3415

In [83]:
df_sub = pd.read_csv("./data/submission_train_else.csv") # 7765
df_test = pd.read_csv('./data/test_with_label.csv')

count_same = [] # 예측이 같은 것의 개수
count_norelation = [] # 그 중 no relation의 개수
for i in pred_else:
    if df_sub.iloc[i,:]['pred_label'] == df_test.iloc[i,:]['label']:
        count_same.append(i)
        if df_sub.iloc[i,:]['pred_label'] == 'no_relation':
            count_norelation.append(i)

print(f'같은 것 개수 : {len(count_same)}')
print(f'그 중 no_relation 의 개수 : {len(count_norelation)}')

print(f'True "else" 중 정확하게 맞춘 개수 : {len(count_same) - len(count_norelation)}')

같은 것 개수 : 2083
그 중 no_relation 의 개수 : 0
True "else" 중 정확하게 맞춘 개수 : 2083


## 29 label ver.

In [75]:
df_else = pd.read_csv('./data/submission_else.csv')

l = pd.DataFrame()
li = []
id_nonexist = set(range(7765)) - set(list(df_else['id']))
for i in id_nonexist:
#     l = pd.concat([l, pd.Series([i, 'no_relation']).T], axis = 0)
    li.append([i, 'no_relation'])
tmp = pd.concat([l, pd.DataFrame(li)], axis=0)
tmp.columns = ['id', 'pred_label']

df_else = pd.concat([df_else[['id', 'pred_label']], tmp], axis = 0)
df_else = df_else.sort_values(by='id')
df_else

,id,pred_label
0,0,org:members
1,1,per:alternate_names
0,2,no_relation
1,3,no_relation
2,4,no_relation
...,...,...
3412,7760,org:place_of_headquarters
4348,7761,no_relation
3413,7762,org:top_members/employees
3414,7763,org:founded


In [76]:
# df_sub = pd.read_csv("./data/submission_no_else_no.csv") # 7765
df_test = pd.read_csv('./data/test_with_label.csv')

count_same = [] # 예측이 같은 것의 개수
count_norelation = [] # 그 중 no relation의 개수
for i in pred_else:
    if df_else.iloc[i,:]['pred_label'] == df_test.iloc[i,:]['label']:
        count_same.append(i)
        if df_else.iloc[i,:]['pred_label'] == 'no_relation':
            count_norelation.append(i)

print(f'같은 것 개수 : {len(count_same)}')
print(f'그 중 no_relation 의 개수 : {len(count_norelation)}')

print(f'True "else" 중 정확하게 맞춘 개수 : {len(count_same) - len(count_norelation)}')

같은 것 개수 : 2042
그 중 no_relation 의 개수 : 0
True "else" 중 정확하게 맞춘 개수 : 2042
